In [ ]:
import pandas as pd
import numpy as np
import os
import random
import shutil
import cv2
import matplotlib.pyplot as plt
import wandb

In [ ]:
wandb.login(key='a303019d2b2e6fab4a9269191c351ee572246477')

In [ ]:
image_path="/kaggle/input/human-faces-object-detection/images"

In [ ]:
bb_data=pd.read_csv("/kaggle/input/human-faces-object-detection/faces.csv")
bb_data.head(10)

In [ ]:
faces=os.listdir(image_path)
random.shuffle(faces)
faces[0:5]

In [ ]:
os.makedirs("/kaggle/working/images/train")
os.makedirs("/kaggle/working/images/valid")
os.makedirs("/kaggle/working/images/test")
os.makedirs("/kaggle/working/labels/train")
os.makedirs("/kaggle/working/labels/valid")
os.makedirs("/kaggle/working/labels/test")
os.makedirs("/kaggle/working/labels/label")

In [ ]:
data_len=len(faces)
train_size=0.8
valid_size=0.1
test_size=0.1

In [ ]:
image_train=faces[:int(data_len*train_size)]
image_valid=faces[int(data_len*train_size):int(data_len*(train_size+valid_size))]
image_test=faces[int(data_len*(train_size+valid_size)):]

In [ ]:
for image in image_train:
    image_path=os.path.join('/kaggle/input/human-faces-object-detection/images',image)
    new_img=cv2.resize(cv2.imread(image_path), (640, 640))
    new_image_path=os.path.join('/kaggle/working/images/train',image)
    cv2.imwrite(new_image_path,new_img)
for image in image_valid:
    image_path=os.path.join('/kaggle/input/human-faces-object-detection/images',image)
    new_img=cv2.resize(cv2.imread(image_path), (640, 640))
    new_image_path=os.path.join('/kaggle/working/images/valid',image)
    cv2.imwrite(new_image_path,new_img)
for image in image_test:
    image_path=os.path.join('/kaggle/input/human-faces-object-detection/images',image)
    new_img=cv2.resize(cv2.imread(image_path), (640, 640))
    new_image_path=os.path.join('/kaggle/working/images/test',image)
    cv2.imwrite(new_image_path,new_img)

In [ ]:
len(image_train),len(image_valid),len(image_test)

In [ ]:
label_train=list(map(lambda x: os.path.splitext(x)[0]+".txt",image_train))
label_valid=list(map(lambda x: os.path.splitext(x)[0]+".txt",image_valid))
label_test=list(map(lambda x: os.path.splitext(x)[0]+".txt",image_test))

In [ ]:
len(label_train),len(label_valid),len(label_test)

In [ ]:
bb_data["class"]=0
bb_data["x_cen"]=0.5*(bb_data["x0"]+bb_data["x1"])
bb_data["y_cen"]=0.5*(bb_data["y0"]+bb_data["y1"])
bb_data["bb_width"]=bb_data["x1"]-bb_data["x0"]
bb_data["bb_height"]=bb_data["y1"]-bb_data["y0"]
bb_data["x_scale"]=bb_data["x_cen"]/bb_data["width"]
bb_data["y_scale"]=bb_data["y_cen"]/bb_data["height"]
bb_data["w_scale"]=bb_data["bb_width"]/bb_data["width"]
bb_data["h_scale"]=bb_data["bb_height"]/bb_data["height"]
bb_data

In [ ]:
bb_group=bb_data.groupby("image_name")

In [ ]:
for image in bb_group:
    img_df=bb_group.get_group(image[0])
    file_name=os.path.splitext(image[0])[0]
    txt_file=file_name+".txt"
    filepath=os.path.join('/kaggle/working/labels/label', txt_file)
    lines=[]
    i=1
    for index,row in img_df.iterrows():
        if i!=len(img_df):
            line=str(row["class"])+" "+str(row['x_scale'])+" "+str(row['y_scale'])+" "+str(row['w_scale'])+" "+str(row['h_scale'])+"\n"
            lines.append(line)
        else:
            line=str(row["class"])+" "+str(row['x_scale'])+" "+str(row['y_scale'])+" "+str(row['w_scale'])+" "+ str(row['h_scale'])
            lines.append(line)
        i=i+1
    with open(filepath, 'w') as file:
        file.writelines(lines)
#     print(image[0])
        

In [ ]:
for label in label_train:
    label_path=os.path.join('/kaggle/working/labels/label',label)
    new_label_path=os.path.join('/kaggle/working/labels/train',label)
    shutil.move(label_path,new_label_path)
for label in label_valid:
    label_path=os.path.join('/kaggle/working/labels/label',label)
    new_label_path=os.path.join('/kaggle/working/labels/valid',label)
    shutil.move(label_path,new_label_path)
for label in label_test:
    label_path=os.path.join('/kaggle/working/labels/label',label)
    new_label_path=os.path.join('/kaggle/working/labels/test',label)
    shutil.move(label_path,new_label_path)

In [ ]:
len(os.listdir('/kaggle/working/labels/label'))

In [ ]:
shutil.rmtree('/kaggle/working/labels/label')

In [ ]:
ln_1='# Train/val/test sets'+"\n"
ln_2='train: ' +"'"+'/kaggle/working/images/train'+"'"+"\n"
ln_3='val: ' +"'" + '/kaggle/working/images/valid'+"'"+"\n"
ln_4='test: ' +"'" + '/kaggle/working/images/test'+"'"+"\n"
ln_5='\n'
ln_6='# Classes'+"\n"
ln_7='names:'+"\n"
ln_8='  0: face'
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8]

In [ ]:
config_path=os.path.join('/kaggle/working', 'config.yaml')
config_path

In [ ]:
with open(config_path, 'w') as f:
    f.writelines(config_lines)

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model=YOLO('yolov5n.yaml')

In [ ]:
#a303019d2b2e6fab4a9269191c351ee572246477
results=model.train(data=config_path, epochs=100, resume=True, iou=0.5, conf=0.001,save=True)

In [ ]:
def draw_bbox(img,annotation):
    x0=int(annotation['x0'])/int(annotation['width'])*640
    x1=int(annotation['x1'])/int(annotation['width'])*640
    y0=int(annotation['y0'])/int(annotation['height'])*640
    y1=int(annotation['y1'])/int(annotation['height'])*640
    bbox=cv2.rectangle(img,(int(x0),int(y0)),(int(x1),int(y1)),(0,255,0),2)
    plt.imshow(bbox)

In [ ]:
k=random.randint(0,len(image_test)-1)
image=image_test[k]
# img_path=os.path.join(imgtrainpath, imgtrain_list[k])
img_df=bb_group.get_group(image)
filepath=os.path.join('/kaggle/working/images/test', image)
image=cv2.imread(filepath)
for index,row in img_df.iterrows():
    draw_bbox(image,row)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
# k=random.randint(0,len(image_test)-1)
image=image_test[k]
results = model(os.path.join('/kaggle/working/images/test',image))  # return a list of Results objects

# img_df=bb_group.get_group(image)
# filepath=os.path.join('/kaggle/working/images/test', image)
# image=cv2.imread(filepath)
for result in results:
    res_plotted = result.plot()
    plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
# for index,row in img_df.iterrows():
#     draw_bbox(image,row)
    

In [ ]:
fig, (axs1,axs2) = plt.subplots(1,2)
k=random.randint(0,len(image_test)-1)
image=image_test[k]
img_df=bb_group.get_group(image)
filepath=os.path.join('/kaggle/working/images/test', image)
results = model(filepath)  # return a list of Results objects
image=cv2.imread(filepath)
for index,row in img_df.iterrows():
    draw_bbox(image,row)
axs1.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    for result in results:
        res_plotted = result.plot()
    axs2.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))